#### Libraries

In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/albayerga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import pandas as pd
import collections
from numpy import linalg as la

#### Load data into memory


1 way

In [11]:
docs_path = 'data/farmers-protest-tweets.json'
with open(docs_path) as fp:
    tweets = fp.readlines()
tweets = [l.strip().replace(' +', ' ') for l in tweets]

In [17]:
print("Total number of farmer tweets of 2021 in the corpus: {}".format(len(tweets)))

Total number of farmer tweets of 2021 in the corpus: 117407


In [18]:
print(tweets[0])

{"url": "https://twitter.com/ArjunSinghPanam/status/1364506249291784198", "date": "2021-02-24T09:23:35+00:00", "content": "The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF", "renderedContent": "The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur twitter.com/ravisinghka/st\u2026", "id": 1364506249291784198, "user": {"username": "ArjunSinghPanam", "displayname": "Arjun Singh Panam", "id": 45091142, "description": "Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie", "rawDescription": "Global

2 way

In [12]:
data_tweets = pd.read_json(docs_path, lines=True)
print(data_tweets.head())

                                                 url  \
0  https://twitter.com/ArjunSinghPanam/status/136...   
1  https://twitter.com/PrdeepNain/status/13645062...   
2  https://twitter.com/parmarmaninder/status/1364...   
3  https://twitter.com/anmoldhaliwal/status/13645...   
4  https://twitter.com/KotiaPreet/status/13645061...   

                       date  \
0 2021-02-24 09:23:35+00:00   
1 2021-02-24 09:23:32+00:00   
2 2021-02-24 09:23:22+00:00   
3 2021-02-24 09:23:16+00:00   
4 2021-02-24 09:23:10+00:00   

                                             content  \
0  The world progresses while the Indian police a...   
1  #FarmersProtest \n#ModiIgnoringFarmersDeaths \...   
2  ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...   
3  @ReallySwara @rohini_sgh watch full video here...   
4  #KisanEktaMorcha #FarmersProtest #NoFarmersNoF...   

                                     renderedContent                   id  \
0  The world progresses while the Indian police a...  13645062

#### Extraer campos Tweet | Date | Hashtags| Likes | Retweets | Url


In [15]:
data_tweets= data_tweets[['content', 'date', 'likeCount', 'retweetCount', 'url']]
data_tweets.head()

,content,date,likeCount,retweetCount,url
0,The world progresses while the Indian police a...,2021-02-24 09:23:35+00:00,0,0,https://twitter.com/ArjunSinghPanam/status/136...
1,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,2021-02-24 09:23:32+00:00,0,0,https://twitter.com/PrdeepNain/status/13645062...
2,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,2021-02-24 09:23:22+00:00,0,0,https://twitter.com/parmarmaninder/status/1364...
3,@ReallySwara @rohini_sgh watch full video here...,2021-02-24 09:23:16+00:00,0,0,https://twitter.com/anmoldhaliwal/status/13645...
4,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,2021-02-24 09:23:10+00:00,0,0,https://twitter.com/KotiaPreet/status/13645061...


In [18]:
for i in range(1):
    print(data_tweets.iloc[i]['content'])

The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. 

@narendramodi @DelhiPolice Shame on you. 

#ModiDontSellFarmers 
#FarmersProtest 
#FreeNodeepKaur https://t.co/es3kn0IQAF


#### Pre-process document

In [68]:
import string
def build_terms(line):

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english")) #the""is""in""and""to" , "change to different languages??"

    line = line.lower()  # everything to lowercase
    translator = str.maketrans('', '', string.punctuation)
    line = line.translate(translator)  # remove punctuation

    line= line.split() # tokenize the text, get a list of terms
    line= [word for word in line if word not in stop_words] # remove stop_words
    line= [stemmer.stem(word) for word in line ] # steam

    return line

In [70]:
terms = build_terms("hello my #name is @john. I am a student, #student")
print(terms)

['hello', 'name', 'john', 'student', 'student']


In [71]:
def preprocess_document(lines):
    for i in range(len(lines)):
       twt_content = lines.iloc[i]['content']
       content_processed = build_terms(twt_content)
       if i == 0:
            print(twt_content)
            print(content_processed)

preprocess_document(data_tweets)

world progresses Indian police Govt still trying take India back horrific past tyranny. @narendramodi @DelhiPolice Shame you. #ModiDontSellFarmers #FarmersProtest #FreeNodeepKaur https://t.co/es3kn0IQAF
['world', 'progress', 'indian', 'polic', 'govt', 'still', 'tri', 'take', 'india', 'back', 'horrif', 'past', 'tyranni', 'narendramodi', 'delhipolic', 'shame', 'modidontsellfarm', 'farmersprotest', 'freenodeepkaur', 'httpstcoes3kn0iqaf']


tweet_document_ids_map

#### Exploratory Data Analysis